## SQLAlchemy

In [1]:
import configparser
from configparser import ConfigParser, ExtendedInterpolation

In [2]:
# instantiate config parser
config = configparser.ConfigParser(interpolation=ExtendedInterpolation())

In [3]:
# read a config file
config.read('../data/config.ini')

['../data/config.ini']

In [4]:
# observe the sections in the config file
config.sections()

['USER', 'DATABASE']

In [5]:
# config uses dict syntax to store values
TEST_DB_PATH = config['DATABASE']['TEST_DB_PATH']
DB_PATH = config['DATABASE']['DB_PATH']

In [6]:
TEST_DB_PATH

'sqlite:///C:\\Users\\alsherman\\Desktop\\General_Assembly\\ds-dc-30\\Lesson18_Databases\\code\\data\\sql\\test.db'

SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language.

In [7]:
from IPython.core.display import display, HTML

sqlalchemy_url = 'http://www.sqlalchemy.org/'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(sqlalchemy_url)
HTML(iframe)

Python's SQLAlchemy and Object-Relational Mapping

A common task when programming any web service is the construction of a solid database backend. In the past, programmers would write raw SQL statements, pass them to the database engine and parse the returned results as a normal array of records. Nowadays, programmers can write Object-relational mapping (ORM) programs to remove the necessity of writing tedious and error-prone raw SQL statements that are inflexible and hard-to-maintain.

ORM is a programming technique for converting data between incompatible type systems in object-oriented programming languages. Usually, the type system used in an OO language such as Python contains types that are non-scalar, namely that those types cannot be expressed as primitive types such as integers and strings

- Source: http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/
- Source: https://www.pythonsheets.com/notes/python-sqlalchemy.html

In [8]:
tutorial_url = 'http://docs.sqlalchemy.org/en/latest/orm/tutorial.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(tutorial_url)
HTML(iframe)

In [9]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

#### Declarative Base
Declarative base maintains a catalog of classes and tables. It keeps track of all the database tables that are accessible through SQLAlchemy.

In [10]:
from sqlalchemy import Column, Text, Integer

#### SQLAlchemy column and data types
Each database field is provided a column and data type in SQLAlchemy

- Column: specifies a column that can read, write and store a particular type of Python data.  
- Text: In SQL, usually corresponds to CLOB or TEXT
- Integer: integer Data type

In [11]:
# provide a class name for the database table
class Sections(Base):
    # provide a name used in SQL to query the table
    __tablename__ = 'SECTIONS'

    # provide column names and data types
    # for each field in the database
    # additional paramater are available for the columns
    # and data types
    section_id = Column(Integer(), primary_key=True, autoincrement=True)
    section_name = Column(Text())
    section_text = Column(Text())

    def __repr__(self):
        return '<Sections section_id: {} | section_name: {}>'.format(
            self.section_id, self.section_name)

#### SQLAlchemy Classes

Each SQLAlchemy class corresponds to a database table.

Within the class, we define details about the table to which we’ll be mapping, primarily the table name, and names and datatypes of columns.

A class using Declarative at a minimum needs a __tablename__ attribute, and at least one Column which is part of a primary key. 

Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

In [12]:
sections = Sections()
sections

<Sections section_id: None | section_name: None>

**_ _repr_ _** (represent) provides the output when using print. This explains why the output is sometimes different when using print  

In [13]:
list_one = ['a','b','c']
list_two = [1,2,3]

print(zip(list_one, list_two))
zip(list_one, list_two)

In [14]:
# confirm test database directory
TEST_DB_PATH

'sqlite:///C:\\Users\\alsherman\\Desktop\\General_Assembly\\ds-dc-30\\Lesson18_Databases\\code\\data\\sql\\test.db'

#### Below are common urls to connect to a variery of databases

SQLAlchemy includes dialects for common databases, including Microsoft SQL Server, MySQL, Oracle, PostgreSQL, SQLite, and more. Additional external dialects are available for other databases like Amazon Redshift.


In [15]:
database_url = 'http://docs.sqlalchemy.org/en/latest/core/engines.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(database_url)
HTML(iframe)

In [16]:
from sqlalchemy import create_engine

# create database tables
engine = create_engine(TEST_DB_PATH)
Base.metadata.create_all(engine)

In this tutorial we use a SQLite database. To connect we use create_engine()

#### Create Engine
The return value of create_engine() is an instance of Engine, and it represents the core interface to the database.

#### Lazy Connecting
The Engine, when first returned by create_engine(), has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database

#### Base.metaData.create_all()
As our SQLite database does not yet have a Sections table present, we can issue a CREATE TABLE statements to the database.

Base.metaData.create_all(), passing in our Engine as a source of database connectivity. We will see that special commands are first emitted to check for the presence of the users table, and following that the actual CREATE TABLE statement

In [17]:
from sqlalchemy.orm import sessionmaker

engine = create_engine(TEST_DB_PATH)
Session = sessionmaker(bind=engine)
session = Session()

#### SQLAlchemy Session

In the most general sense, the Session establishes all conversations with the database and represents a “holding zone” for all the objects which you’ve loaded or associated with it during its lifespan. It provides the entrypoint to acquire a Query object, which sends queries to the database using the Session object’s current database connection, populating result rows into objects that are then stored in the Session

All changes to objects maintained by a Session are tracked - before the database is queried again or before the current transaction is committed, it flushes all pending changes to the database. This is known as the Unit of Work pattern.

#### Insert data into the database

In [18]:
section = Sections(
      section_name = 'first section'
    , section_text = 'text from the first section'
)

session.add(section)

At this point, we say that the instance is pending; no SQL has yet been issued and the object is not yet represented by a row in the database. 

The Session will issue the SQL to persist the section as soon as is needed, using a process known as a flush. If we query the database for the section, all pending information will first be flushed, and the query is issued immediately thereafter.

In [19]:
# commit (save) all annual reports to the database
session.commit()

#### Commit

We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit().

commit() flushes the remaining changes to the database, and commits the transaction. The connection resources referenced by the session are now returned to the connection pool. Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed.

In [20]:
# query database to check if data was added
result = session.query(Sections).all()
for row in result:
    print('section_id: {}'.format(row.section_id))
    print('section_name: {}'.format(row.section_name))
    print('section_text: {}'.format(row.section_text))

section_id: 1
section_name: first section
section_text: text from the first section


In [21]:
# view the __repr__ of the SQLAlchemy class
result

[<Sections section_id: 1 | section_name: first section>]

In [22]:
# Delete data from Sections to reset table (optional)
session.execute("DELETE FROM Sections")
session.commit()

## Exercise

Create and query a data containing StackOverflow questions data.

The questions are randomized and contains a mix of verbose text sites as well as sites related to math and programming. The number of questions from each site may vary, and no filtering has been performed on the questions (such as closed questions).

#### Data Dictionary:
- Id - Unique identifier for each question
- Title - The question's title
- Body - The body of the question
- Tags - The tags associated with the question (all lowercase, should not contain tabs '\t' or ampersands '&')

Source: https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction

### Create a table named StackOverflow with the following columns and data types
- tablename = 'STACKOVERFLOW'
- Id - Integer, primary_key
- Title - Text, nullble
- Body - Text, nullable
- Tags - Text, nullable
- Add a __repr__ that prints out the Id and Title

In [23]:
class StackOverflow(Base):
    __tablename__ = 'STACKOVERFLOW'

    Id = Column(Integer(), nullable=False, primary_key=True)
    Title = Column(Text(), nullable=True)
    Body = Column(Text(), nullable=True)
    Tags = Column(Text(), nullable=True) 
    
    def __repr__(self):
        return '<StackOverflow Id: {} | Title: {}>'.format(
            self.Id, self.Title)

In [24]:
# confirm the path to the stackoverflow database
DB_PATH

'sqlite:///C:\\Users\\alsherman\\Desktop\\General_Assembly\\ds-dc-30\\Lesson18_Databases\\code\\data\\sql\\stackoverflow.sqlite'

In [25]:
# create a engine session to connect to the database 
from sqlalchemy.orm import sessionmaker

engine = create_engine(DB_PATH)
Session = sessionmaker(bind=engine)
session = Session()

# SQL

In [27]:
database_url = 'http://docs.sqlalchemy.org/en/latest/orm/tutorial.html'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(database_url)
HTML(iframe)

In [28]:
# SQLAlchemy ORM
for result in session.query(StackOverflow)[0:3]:
    print('Question Id: {}'.format(result.Id))
    print('TITLE: {}'.format(result.Title))
    print('TAGS: {} \n'.format(result.Tags))

Question Id: 1
TITLE: How to check if an uploaded file is an image without mime type?
TAGS: php image-processing file-upload upload mime-types 

Question Id: 2
TITLE: How can I prevent firefox from closing when I press ctrl-w
TAGS: firefox 

Question Id: 3
TITLE: R Error Invalid type (list) for variable
TAGS: r matlab machine-learning 



In [29]:
# SQLAlchamy Core - SQL Expression Language
from sqlalchemy.sql import text

query = text('SELECT * FROM StackOverflow LIMIT 3')
for result in session.execute(query):
    print('Question Id: {}'.format(result.Id))
    print('TITLE: {}'.format(result.Title))
    print('TAGS: {} \n'.format(result.Tags))

Question Id: 1
TITLE: How to check if an uploaded file is an image without mime type?
TAGS: php image-processing file-upload upload mime-types 

Question Id: 2
TITLE: How can I prevent firefox from closing when I press ctrl-w
TAGS: firefox 

Question Id: 3
TITLE: R Error Invalid type (list) for variable
TAGS: r matlab machine-learning 



In [30]:
# Pandas
import pandas as pd
pd.read_sql('SELECT * FROM STACKOVERFLOW LIMIT 3', con=engine)

,index,Id,Title,Body,Tags
0,0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...
1,1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox
2,2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning


### SELECT

In [31]:
# SQLAlchemy ORM
session.query(StackOverflow.Id, StackOverflow.Tags).limit(3).all()

[(1, 'php image-processing file-upload upload mime-types'),
 (2, 'firefox'),
 (3, 'r matlab machine-learning')]

In [32]:
# Raw SQL Query
print(session.query(StackOverflow.Id, StackOverflow.Tags).limit(3))

SELECT "STACKOVERFLOW"."Id" AS "STACKOVERFLOW_Id", "STACKOVERFLOW"."Tags" AS "STACKOVERFLOW_Tags" 
FROM "STACKOVERFLOW"
 LIMIT ? OFFSET ?


In [33]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Id, Tags FROM StackOverflow LIMIT 3')
session.execute(query).fetchall()

[(1, 'php image-processing file-upload upload mime-types'),
 (2, 'firefox'),
 (3, 'r matlab machine-learning')]

In [34]:
#Pandas 
pd.read_sql('SELECT Id, Tags FROM STACKOVERFLOW LIMIT 3', con=engine)

,Id,Tags
0,1,php image-processing file-upload upload mime-t...
1,2,firefox
2,3,r matlab machine-learning


### WHERE

In [35]:
# SQLAlchemy ORM
session.query(StackOverflow.Id, StackOverflow.Title
    ).filter(StackOverflow.Tags == 'python').limit(3).all()

[(206, 'Splitting a list'),
 (990, 'Split based by a-z character in an alphanumeric string in python'),
 (1194, 'How to : Python Convert Bytes to Readable ASCII/Unicode')]

In [36]:
# Raw SQL Query
print(session.query(StackOverflow.Id, StackOverflow.Title
    ).filter(StackOverflow.Tags == 'python').limit(3))

SELECT "STACKOVERFLOW"."Id" AS "STACKOVERFLOW_Id", "STACKOVERFLOW"."Title" AS "STACKOVERFLOW_Title" 
FROM "STACKOVERFLOW" 
WHERE "STACKOVERFLOW"."Tags" = ?
 LIMIT ? OFFSET ?


In [37]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Id, Tags, Title FROM STACKOVERFLOW WHERE Tags = "python" LIMIT 3')
session.execute(query).fetchall()

[(206, 'python', 'Splitting a list'),
 (990, 'python', 'Split based by a-z character in an alphanumeric string in python'),
 (1194, 'python', 'How to : Python Convert Bytes to Readable ASCII/Unicode')]

In [38]:
# Pandas
pd.read_sql('SELECT Id, Tags, Title FROM STACKOVERFLOW WHERE Tags = "python" LIMIT 3', con=engine)

,Id,Tags,Title
0,206,python,Splitting a list
1,990,python,Split based by a-z character in an alphanumeri...
2,1194,python,How to : Python Convert Bytes to Readable ASCI...


### LIKE

In [39]:
# SQLAlchemy ORM
session.query(StackOverflow.Tags
    ).filter(StackOverflow.Tags.like('%python%')).limit(3).all()

[('c# java c++ python strong-typing'),
 ('python encryption'),
 ('c++ python swig')]

In [40]:
# Raw SQL Query
print(session.query(StackOverflow.Tags
    ).filter(StackOverflow.Tags.like('%python%')).limit(3))

SELECT "STACKOVERFLOW"."Tags" AS "STACKOVERFLOW_Tags" 
FROM "STACKOVERFLOW" 
WHERE "STACKOVERFLOW"."Tags" LIKE ?
 LIMIT ? OFFSET ?


In [41]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Tags FROM STACKOVERFLOW WHERE Tags LIKE "%python%" LIMIT 3')
session.execute(query).fetchall()

[('c# java c++ python strong-typing',),
 ('python encryption',),
 ('c++ python swig',)]

In [42]:
# Pandas
pd.read_sql('SELECT Tags FROM STACKOVERFLOW WHERE Tags LIKE "%python%" LIMIT 3', con=engine)

,Tags
0,c# java c++ python strong-typing
1,python encryption
2,c++ python swig


### IN

In [43]:
# SQLAlchemy ORM
tags = ['python', 'java', 'sql']
session.query(StackOverflow.Title
    ).filter(StackOverflow.Tags.in_(tags)).limit(3).first()

('Splitting a list')

In [44]:
# Raw SQL Query
print(session.query(StackOverflow.Title
    ).filter(StackOverflow.Tags.in_(['python', 'java', 'sql'])).limit(3))

SELECT "STACKOVERFLOW"."Title" AS "STACKOVERFLOW_Title" 
FROM "STACKOVERFLOW" 
WHERE "STACKOVERFLOW"."Tags" IN (?, ?, ?)
 LIMIT ? OFFSET ?


In [45]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT Title FROM STACKOVERFLOW WHERE Tags IN ("python","java","sql") LIMIT 3')
session.execute(query).fetchall()

[('Splitting a list',),
 ('java.util.UUID.fromString not checking length',),
 ('Split based by a-z character in an alphanumeric string in python',)]

In [46]:
# Pandas
pd.read_sql('SELECT Title FROM STACKOVERFLOW WHERE Tags IN ("python","java","sql") LIMIT 3'
            , con=engine)

,Title
0,Splitting a list
1,java.util.UUID.fromString not checking length
2,Split based by a-z character in an alphanumeri...


### COUNT

In [47]:
# SQLAlchemy ORM
session.query(StackOverflow).count()

30000

In [48]:
# SQLAlchamy Core - SQL Expression Language
query = text('SELECT COUNT(*) FROM STACKOVERFLOW')
session.execute(query).scalar()  # scalar returns a single value (no tuple)

30000

In [49]:
# Pandas
pd.read_sql('SELECT COUNT(*) FROM STACKOVERFLOW', con=engine)

,COUNT(*)
0,30000


### GROUP BY

In [50]:
# SQLAlchemy ORM
from sqlalchemy import func

session.query(StackOverflow.Tags, func.count(StackOverflow.Tags)
    ).group_by(StackOverflow.Tags
    ).order_by(func.count(StackOverflow.Tags).desc()
    ).limit(3).all()

[('android', 241), ('javascript', 148), ('php', 142)]

In [51]:
# Raw SQL Query
print(session.query(StackOverflow.Tags, func.count(StackOverflow.Tags)
    ).group_by(StackOverflow.Tags
    ).order_by(func.count(StackOverflow.Tags).desc()
    ).limit(3))

SELECT "STACKOVERFLOW"."Tags" AS "STACKOVERFLOW_Tags", count("STACKOVERFLOW"."Tags") AS count_1 
FROM "STACKOVERFLOW" GROUP BY "STACKOVERFLOW"."Tags" ORDER BY count("STACKOVERFLOW"."Tags") DESC
 LIMIT ? OFFSET ?


In [52]:
# SQLAlchamy Core - SQL Expression Language
query = text("""SELECT Tags, COUNT(Tags)
            FROM STACKOVERFLOW
            GROUP BY Tags
            ORDER BY COUNT(Tags) Desc
            LIMIT 3""")
session.execute(query).fetchall()

[('android', 241), ('javascript', 148), ('php', 142)]

In [53]:
# Pandas
pd.read_sql("""SELECT Tags, COUNT(Tags)
            FROM STACKOVERFLOW
            GROUP BY Tags
            ORDER BY COUNT(Tags) Desc
            LIMIT 30
            """, con=engine)

,Tags,COUNT(Tags)
0,android,241
1,javascript,148
2,php,142
3,jquery,125
4,javascript jquery,118
5,java,109
6,c#,108
7,python,88
8,mysql,83
9,php mysql,77


## Extra Material

#### determine the schema of an existing table

In [54]:
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection

insp = reflection.Inspector.from_engine(engine)
for d in insp.get_columns('StackOverflow'):
    print('name: {} | type: {} | nullable: {} | primary_key: {}'.format(
        d['name'], d['type'], d['nullable'], d['primary_key']))

name: index | type: BIGINT | nullable: True | primary_key: 0
name: Id | type: BIGINT | nullable: True | primary_key: 0
name: Title | type: TEXT | nullable: True | primary_key: 0
name: Body | type: TEXT | nullable: True | primary_key: 0
name: Tags | type: TEXT | nullable: True | primary_key: 0


#### load database iteratively from pandas

In [55]:
import pandas as pd
import datetime as dt
import zipfile
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [50]:
# Change these to your zip and file to load
ZIP_FILE_NAME = r'PATH_TO_ZIP'
CSV_NAME = r'FILENAME'

start = dt.datetime.now()
chunksize = 30000
zf = zipfile.ZipFile(ZIP_FILE_NAME)

engine = create_engine(DB_PATH)
for ind, df in enumerate(pd.read_csv(
    zf.open(CSV_NAME), chunksize=chunksize, iterator=True, encoding='utf-8', nrows=30_000)):
    
    print('{} seconds: completed {} rows'.format(
        (dt.datetime.now() - start).seconds, ind*chunksize))

    df.to_sql('STACKOVERFLOW', con=engine, if_exists='replace')
 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\alsherman\\Downloads\\Train.zip'

In [ ]:
# Optional - Delete Table
#session.execute("DELETE FROM STACKOVERFLOW")
#session.commit()